In [3]:
import re
import requests
from datetime import datetime
from synced_lyrics import *

In [36]:
def get_meta_data(song, artist, album):
    # song = song.replace(' ', '_')
    # artist = artist.replace(' ', '_')
    # album = album.replace(' ', '_')

    url = f"https://lrclib.net/api/search?artist_name={artist.replace(' ', '_')}&track_name={song.replace(' ', '_')}&albumName={album.replace(' ', '_')}"
    response = requests.get(url)
    print(url)

    if response.status_code == 200:
        for r in response.json():
            if r['name'].lower() == song.lower():
                print('name')
                if r['artistName'].lower() == artist.lower():
                    print('artist')
                    if r['syncedLyrics'] is not None:
                        return r
                else:
                    print("not artsit", r['artist'].lower())
            else:
                print('not name', r['name'].lower(), song.lower())
            
        return "song not found"
    else:
        return "song not found"
    
get_meta_data('Eyes on you', 'seventeen', "spill the feels'")

https://lrclib.net/api/search?artist_name=seventeen&track_name=Eyes_on_you&albumName=spill_the_feels'
name
artist


{'id': 14471802,
 'name': 'Eyes on you',
 'trackName': 'Eyes on you',
 'artistName': 'SEVENTEEN',
 'albumName': 'null',
 'duration': 175.0,
 'instrumental': False,
 'plainLyrics': "Eyes on you, eyes on me\n너의 손을 꼭 잡을 때\nEyes on you, eyes on me\n널 쉴 틈 없이 알고 싶어\n\n섬세한 한마디 우리의 대화\nSo sweet, 시작해 (ah)\n같이 걸어 볼까 under the moonlight? (Moonlight)\nNo matter what you do, I promise you, oh, yeah\n\n같은 BPM 맞춰지는 heart rate\n솔직한 우리 터져 나올 것 같아\n고개를 돌리며 마주 봐 highlight (ooh)\n\nEyes on you, eyes on me\n너의 손을 꼭 잡을 때\nEyes on you, eyes on me\n기적이 나타날 거야\n\nTell me more about yourself\n너의 첫 번째 상처를 내게 줘\nTell me more about yourself\n널 쉴 틈 없이 알고 싶어\n\n이유 없는 대화 너라서 좋아\nSo sweet, 계속해 (ah)\n이런 우리라도 넌 괜찮다면 (yeah)\nNo matter what you do, I promise you, yeah\n\n우리 둘 시선의 끝\n맞닿을 때 생겨나네 피어나는 불\n걷잡을 수 없어, don't matter\n서로가 서로를 꽉 채워\n\nDance with me like it is the last time\nWho let the dogs out after bedtime?\nCan't stop the shindig 'til one hundred degrees\nYou set me free\n\n같은 BPM 맞춰지는 heart rate\n솔직한 우리 터져 나올 것 

In [53]:
def get_synced_lyrics(song):
    synced_lyrics = ''

    file_path = "../official_times/"+ song + ".srt"
    try:
        with open(file_path, 'r') as f:
            synced_lyrics = f.read()
            print("file found")
            return synced_lyrics
        
    except FileNotFoundError:
        print('file not found', file_path)
        return FileNotFoundError


In [54]:
def getDelta(line, replaceChar):
    time_format = "%M:%S:%f"

    start = datetime.strptime(line['start'].replace(replaceChar, ':'), time_format)
    end = datetime.strptime(line['end'].replace(replaceChar, ':'), time_format)

    delta = str(end-start)
    pattern = r"\d{1}:(\d{2}:\d{2}(?:\.\d{2})?)"
    delta = re.findall(pattern, delta)[0]

    if "." not in delta:
        delta += ".00"
    
    delta = delta.replace('.', ':')
    return delta
    

In [55]:
def calc_new_delta(delta, addedTime):
    time_format = "%M:%S:%f"
    old_delta = datetime.strptime(delta, time_format)

    new_delta = old_delta + addedTime
    pattern = r"\d{1}:(\d{2}:\d{2}(?:\.\d{2})?)"
    delta = re.findall(pattern, new_delta)[0]

    if "." not in delta:
        delta += ".00"
    
    delta = delta.replace('.', ':')
    return delta

In [56]:
def clean_official_lyrics(lyrics_raw):
    pattern = r"\d{2}:(\d{2}:\d{2},\d{2})\d{1} --> \d{2}:(\d{2}:\d{2},\d{2})\d{1}\s*([\s\S]*?)(?=\n\d+\n\d{2}:|$)"

    parsed_synced_lyrics = re.findall(pattern, lyrics_raw)
    print(parsed_synced_lyrics)

    new_synced_lyrics = []
    for line in parsed_synced_lyrics:
        start = line[0].replace(',', ':')
        end = line[1].replace(',', ':')
        newLine = {'start': start, 'end': end, 'lyric': line[2].replace('\n', ' ').strip()}
        newLine["delta"] = getDelta(newLine, ',')
        
        new_synced_lyrics.append(newLine)
    return new_synced_lyrics

In [57]:
def clean_lrclib_lyrics(lyrics_raw, duration):
    pattern = r"\[(\d{2}:\d{2}\.\d{2})\]\s*(.*?)(?=\[|$)"

    lyrics_raw = lyrics_raw.replace('\n', '')
    parsed_synced_lyrics = re.findall(pattern, lyrics_raw)

    new_synced_lyrics = []

    for i, line in enumerate(parsed_synced_lyrics):
            lyric = line[1]
            start = line[0].replace('.', ':')
            end = ''

            if lyric == '':
                continue

            if i + 1 >= len(lyrics_raw):
                    minutes = int(duration/60)
                    seconds = int(duration) % 60
                    tenths = int(duration % 1 *10)
                    end = f"{minutes:02}:{seconds:02}:{tenths:02}"
            else:
                end = parsed_synced_lyrics[i+1][0].replace('.', ':')

            newLine = {'start': start, 'end': end, 'lyric': lyric}
            newLine["delta"] = getDelta(newLine, '.')

            new_synced_lyrics.append(newLine)
            
    return new_synced_lyrics

In [84]:
def get_song_info(song, artist, album):
    meta_data = get_meta_data(song, artist, album)

    if meta_data == "song not found":
        print("song not found")
        return
    
    lyrics_success = get_synced_lyrics(song)

    meta_data['syncedLyrics'] = clean_lrclib_lyrics(meta_data['syncedLyrics'],  meta_data['duration'])
    meta_data['source'] = 'lrclib'

    mv_synced = []
    if lyrics_success is not FileNotFoundError:
        mv_synced = clean_official_lyrics(lyrics_success)
        meta_data['source'] = 'official'

        mv_start = time_to_sec(mv_synced[0]['start'])
        lib_start = time_to_sec(meta_data['syncedLyrics'][0]['start'])
        dif = mv_start - lib_start
        print(mv_start, lib_start, dif)

        if dif > 3:
            print('update times')
            for l in mv_synced:
                # print(l)
                old_start = time_to_sec(l['start']) - dif
                old_end = time_to_sec(l['end']) - dif
                l['start'] = decimal_sec_to_time_str(old_start)
                l['end'] = decimal_sec_to_time_str(old_end)
                # print(l)
                # print()
        meta_data['syncedLyrics'] = mv_synced
        
        
    return meta_data

In [59]:
dif = 20

for l in meta_data['syncedLyrics']:
    print(l)
    start_time = time_to_sec(l['start'])
    start_time = start_time - dif
    print(start_time)

{'start': '00:00:82', 'end': '00:06:12', 'lyric': '어쩌다 보니 처음으로 마주하는 오늘이라서', 'delta': '00:05:30'}
-19.18
{'start': '00:06:12', 'end': '00:12:88', 'lyric': '사무치게 아픈 말 한마디에 내가 더 싫어도', 'delta': '00:06:76'}
-13.879999999999999
{'start': '00:12:88', 'end': '00:16:36', 'lyric': '신경 쓰지 말자, 우리 목소리로', 'delta': '00:03:48'}
-7.119999999999999
{'start': '00:16:36', 'end': '00:27:55', 'lyric': '어디서라도 부르자, 청춘찬가', 'delta': '00:11:19'}
-3.6400000000000006
{'start': '00:27:55', 'end': '00:48:36', 'lyric': 'One, two, one, two, three, four', 'delta': '00:20:81'}
7.550000000000001
{'start': '00:48:36', 'end': '00:52:11', 'lyric': '벨 소리가 울릴 때 겁이 나', 'delta': '00:03:75'}
28.36
{'start': '00:52:11', 'end': '00:57:47', 'lyric': '심장이 먼저, 놀라는 요즘', 'delta': '00:05:36'}
32.11
{'start': '00:57:47', 'end': '01:02:69', 'lyric': '혼자 지내고 싶고, 혼자 있기 싫고', 'delta': '00:05:22'}
37.47
{'start': '01:02:69', 'end': '01:05:86', 'lyric': '나도 날 모르겠어', 'delta': '00:03:17'}
42.69
{'start': '01:05:86', 'end': '01:10:65', 'lyric': '도

In [85]:
song = 'cheers to youth'
artist = 'seventeen'
album = "SEVENTEEN 4th Album Repackage ‘SECTOR 17'"

meta_data = get_meta_data(song, artist, album)
print(meta_data)

lyrics_success = get_synced_lyrics(song)

meta_data['syncedLyrics'] = clean_lrclib_lyrics(meta_data['syncedLyrics'],  meta_data['duration'])

meta_data['source'] = 'lrclib'

{'id': 8996815, 'name': 'Cheers to youth', 'trackName': 'Cheers to youth', 'artistName': 'SEVENTEEN', 'albumName': 'SEVENTEEN', 'duration': 192.0, 'instrumental': False, 'plainLyrics': '어쩌다 보니 처음으로 마주하는 오늘이라서\n사무치게 아픈 말 한마디에 내가 더 싫어도\n신경 쓰지 말자, 우리 목소리로\n어디서라도 부르자, 청춘찬가\n\nOne, two, one, two, three, four\n\n벨 소리가 울릴 때 겁이 나\n심장이 먼저, 놀라는 요즘\n혼자 지내고 싶고, 혼자 있기 싫고\n나도 날 모르겠어\n\n도대체 나의 행복은 어디 있나요?\n그 누구도 대답해 줄 수 없어\n꺼진 폰 화면 속에 비친 내 모습\n보며 말할래\n\n오늘 집에 가는 길에 내게 수고했다고\n마냥 쉽지 않았지만 나쁘지 않았다고\n숨 막히는 세상 속에\n작은 것 하나에 잠깐 웃었다고\n\n어쩌다 보니 처음으로 마주하는 오늘이라서\n사무치게 아픈 말 한마디에 내가 더 싫어도\n신경 쓰지 말자, 우리 목소리로\n어디서라도 부르자, 청춘찬가\n\n나의, 나의, 나의\n나의, 나의, 나의\n날 감싸준 포근한 이불 이 사소한 따뜻함에\n또 내일을 기다리면서 잠들게\n\n내일 아침에 울리는 시끄러운 알람이\n어제보단 조금만이라도 밉지 않기를\n숨 막히는 세상 속에\n이 모든 게 나라서 참 좋을 거야\n\n어쩌다 보니 처음으로 마주하는 내일이라도\n사무치게 아픈 말 한마디에 내가 더 싫어도\n신경 쓰지 말자, 우리 목소리로\n어디서라도 부르자, 청춘찬가', 'syncedLyrics': '[00:00.82] 어쩌다 보니 처음으로 마주하는 오늘이라서\n[00:06.12] 사무치게 아픈 말 한마디에 내가 더 싫어도\n[00:12.88] 신경 쓰지 말자, 우리 목소리로\n[00:16.36] 어디서라도 부르자, 청춘찬가\n[00:27.55] One, tw

In [61]:
print(meta_data['syncedLyrics'])

[{'start': '00:00:82', 'end': '00:06:12', 'lyric': '어쩌다 보니 처음으로 마주하는 오늘이라서', 'delta': '00:05:30'}, {'start': '00:06:12', 'end': '00:12:88', 'lyric': '사무치게 아픈 말 한마디에 내가 더 싫어도', 'delta': '00:06:76'}, {'start': '00:12:88', 'end': '00:16:36', 'lyric': '신경 쓰지 말자, 우리 목소리로', 'delta': '00:03:48'}, {'start': '00:16:36', 'end': '00:27:55', 'lyric': '어디서라도 부르자, 청춘찬가', 'delta': '00:11:19'}, {'start': '00:27:55', 'end': '00:48:36', 'lyric': 'One, two, one, two, three, four', 'delta': '00:20:81'}, {'start': '00:48:36', 'end': '00:52:11', 'lyric': '벨 소리가 울릴 때 겁이 나', 'delta': '00:03:75'}, {'start': '00:52:11', 'end': '00:57:47', 'lyric': '심장이 먼저, 놀라는 요즘', 'delta': '00:05:36'}, {'start': '00:57:47', 'end': '01:02:69', 'lyric': '혼자 지내고 싶고, 혼자 있기 싫고', 'delta': '00:05:22'}, {'start': '01:02:69', 'end': '01:05:86', 'lyric': '나도 날 모르겠어', 'delta': '00:03:17'}, {'start': '01:05:86', 'end': '01:10:65', 'lyric': '도대체 나의 행복은 어디 있나요?', 'delta': '00:04:79'}, {'start': '01:10:65', 'end': '01:15:13', 'lyric': '그 누구도

In [82]:
mv_synced = []
if lyrics_success is not FileNotFoundError:
    mv_synced = clean_official_lyrics(lyrics_success)
    meta_data['source'] = 'official'

    mv_start = time_to_sec(mv_synced[0]['start'])
    lib_start = time_to_sec(meta_data['syncedLyrics'][0]['start'])
    dif = mv_start - lib_start
    print(mv_start, lib_start, dif)

    if dif > 3:
        print('update times')
        for l in mv_synced:
            print(l)
            old_start = time_to_sec(l['start']) - dif
            old_end = time_to_sec(l['end']) - dif
            l['start'] = decimal_sec_to_time_str(old_start)
            l['end'] = decimal_sec_to_time_str(old_end)
            print(l)
            print()

[('00:17,26', '00:22,73', '어쩌다 보니 처음으로 마주하는 오늘이라서\n'), ('00:22,81', '00:29,40', '사무치게 아픈 말 한마디에 내가 더 싫어도\n'), ('00:29,40', '00:31,61', '신경 쓰지 말자\n'), ('00:31,69', '00:35,61', '우리 목소리로 어디서라도 부르자\n'), ('00:35,74', '00:37,74', '청춘찬가\n'), ('00:44,16', '00:46,37', '1 2, 1 2 3 4!\n'), ('01:04,85', '01:08,56', '벨 소리가 울릴 때 겁이 나\n'), ('01:08,65', '01:13,94', '심장이 먼저 놀라는 요즘\n'), ('01:14,11', '01:19,20', '혼자 지내고 싶고 혼자 있기 싫고\n'), ('01:19,28', '01:22,41', '나도 날 모르겠어\n'), ('01:22,58', '01:27,08', '도대체 나의 행복은 어디 있나요\n'), ('01:27,17', '01:31,54', '그 누구도 대답해 줄 수 없어\n'), ('01:31,71', '01:41,18', '꺼진 폰 화면 속에 비친 내 모습 보며 말할래\n'), ('01:41,22', '01:45,73', '오늘 집에 가는 길에 내게 수고했다고\n'), ('01:45,81', '01:51,06', '마냥 쉽지 않았지만 나쁘지 않았다고\n'), ('01:51,15', '01:54,98', '숨 막히는 세상 속에\n'), ('01:55,24', '01:58,91', '작은 것 하나에 잠깐 웃었다고\n'), ('01:58,95', '02:04,41', '어쩌다 보니 처음으로 마주하는 오늘이라서\n'), ('02:04,41', '02:10,88', '사무치게 아픈 말 한마디에 내가 더 싫어도\n'), ('02:10,96', '02:13,21', '신경 쓰지 말자\n'), ('02:13,34', '02:17,26', '우리 목소리로 어디서라도 

In [86]:
synced_lyrics = meta_data['syncedLyrics']

for s in synced_lyrics:
    print(s)

{'start': '00:00:82', 'end': '00:06:12', 'lyric': '어쩌다 보니 처음으로 마주하는 오늘이라서', 'delta': '00:05:30'}
{'start': '00:06:12', 'end': '00:12:88', 'lyric': '사무치게 아픈 말 한마디에 내가 더 싫어도', 'delta': '00:06:76'}
{'start': '00:12:88', 'end': '00:16:36', 'lyric': '신경 쓰지 말자, 우리 목소리로', 'delta': '00:03:48'}
{'start': '00:16:36', 'end': '00:27:55', 'lyric': '어디서라도 부르자, 청춘찬가', 'delta': '00:11:19'}
{'start': '00:27:55', 'end': '00:48:36', 'lyric': 'One, two, one, two, three, four', 'delta': '00:20:81'}
{'start': '00:48:36', 'end': '00:52:11', 'lyric': '벨 소리가 울릴 때 겁이 나', 'delta': '00:03:75'}
{'start': '00:52:11', 'end': '00:57:47', 'lyric': '심장이 먼저, 놀라는 요즘', 'delta': '00:05:36'}
{'start': '00:57:47', 'end': '01:02:69', 'lyric': '혼자 지내고 싶고, 혼자 있기 싫고', 'delta': '00:05:22'}
{'start': '01:02:69', 'end': '01:05:86', 'lyric': '나도 날 모르겠어', 'delta': '00:03:17'}
{'start': '01:05:86', 'end': '01:10:65', 'lyric': '도대체 나의 행복은 어디 있나요?', 'delta': '00:04:79'}
{'start': '01:10:65', 'end': '01:15:13', 'lyric': '그 누구도 대답해 줄 수 없어